In [12]:
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

import matplotlib as plt

In [13]:
output_size = 1         # sonraki N degeri tahmin etmeye calis
epochs = 100            # islemin tekrar sayisi
features = 5            # input kaç birbirini tekrar eden veriden oluşacak

In [14]:
def get_data(y):
    # lstm datayi (rows, timesteps, features) formatinda ister
    # rows : ornek sayimiz
    # timesteps : zaman adimlari
    # features : ogrenme verimizin sutun sayisi
    # training verisi (rows, 1, features)
    # output verisi (rows, output_size)

    train_size = int(len(y)*0.7)                # Verinin %70 ini egitim icin kalaninin da test icin ayiracagiz, 180


    ############## TRAIN DATA ####################
    train_x = []
    train_y = []
    for i in range(0, train_size - features - output_size):
        tmp_x = y[i:(i+features)]
        tmp_y = y[(i+features):(i+features+output_size)]
        train_x.append(np.reshape(tmp_x, (1, features)))
        train_y.append(tmp_y)

    train_x = np.array(train_x)
    train_y = np.array(train_y)

    ########### TEST DATA ########################
    test_x = []
    test_y = []
    last = len(y) - output_size - features
    for i in range(train_size, last):
        tmp_x = y[i:(i+features)]
        tmp_y = y[(i + features):(i + features + output_size)]
        test_x.append(np.reshape(tmp_x, (1, features)))
        test_y.append(tmp_y)

    test_x = np.array(test_x)
    test_y = np.array(test_y)

    ######## Tahmin edilecek data #######################
    data_x = []
    tmp_x = y[-features:len(y)]
    data_x.append(np.reshape(tmp_x, (1, features)))
    data_x = np.array(data_x)

    return train_x, train_y, test_x, test_y, data_x

In [15]:
raw_data = pd.read_csv('./dataset/uev.txt', header=None, names=["i", "t", "y"])
t = np.array(raw_data.t.values)
y = np.array(raw_data.y.values)

min = y.min()
max = y.max()

y = np.interp(y, (min, max), (-1, +1))

x_train, y_train, x_test, y_test, data_x = get_data(y)

In [16]:
data_x

array([[[0.55007526, 0.55053838, 0.55123307, 0.54590714, 0.55378025]]])

In [17]:
model = Sequential()
model.add(LSTM(5, input_shape=(1, features), return_sequences=True))
model.add(Dropout(0.25))
model.add(LSTM(7))
model.add(Dense(output_size))

model.compile(loss='mse', optimizer='rmsprop', metrics=['mae'])
model.fit(x_train, y_train, epochs=epochs, verbose=1, batch_size=4)

score = model.evaluate(x_test, y_test)
print("%2s: %.2f%%" % (model.metrics_names[1], score[1]*100))


Epoch 1/100
43/43 [==============================] - 3s 2ms/step - loss: 0.2562 - mae: 0.4004
Epoch 2/100
43/43 [==============================] - 0s 2ms/step - loss: 0.2330 - mae: 0.3725
Epoch 3/100
43/43 [==============================] - 0s 2ms/step - loss: 0.2195 - mae: 0.3649
Epoch 4/100
43/43 [==============================] - 0s 2ms/step - loss: 0.1948 - mae: 0.3483
Epoch 5/100
43/43 [==============================] - 0s 2ms/step - loss: 0.1515 - mae: 0.3104
Epoch 6/100
43/43 [==============================] - 0s 2ms/step - loss: 0.1013 - mae: 0.2438
Epoch 7/100
43/43 [==============================] - 0s 2ms/step - loss: 0.0750 - mae: 0.1904
Epoch 8/100
43/43 [==============================] - 0s 2ms/step - loss: 0.0494 - mae: 0.1648
Epoch 9/100
43/43 [==============================] - 0s 2ms/step - loss: 0.0443 - mae: 0.1520
Epoch 10/100
43/43 [==============================] - 0s 2ms/step - loss: 0.0365 - mae: 0.1374
Epoch 11/100
43/43 [==============================] - 0s 2m

In [18]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 1, 5)              220       
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 5)              0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 7)                 364       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 8         
Total params: 592
Trainable params: 592
Non-trainable params: 0
_________________________________________________________________


In [19]:
data_y = model.predict(data_x)

result = np.interp(data_y, (-1, +1), (min, max))

print("Gelecekteki Degerler (output_size) :", result)

Gelecekteki Degerler (output_size) : [[4.22476053]]
